In [ ]:
# SPDX-License-Identifier: Apache-2.0 AND CC-BY-NC-4.0
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Quantum Enhanced Optimization for Radar and Communications Applications 


The Low Autocorrelation Binary Sequences (LABS) is an important and challenging optimization problem with applications related to radar, telecommunications, and other signal related applications. This CUDA-Q Academic module will focus on a clever quantum-enhanced hybrid method developed in a collaboration between Kipu Quantum, University of the Basque Country EHU, and NVIDIA for solving the LABS problem. (This notebook was jointly developed with input from all collaborators.)

Other CUDA-Q Academic modules like [Divide and Conquer MaxCut QAOA](https://github.com/NVIDIA/cuda-q-academic/tree/main/qaoa-for-max-cut) and [Quantum Finance](https://github.com/NVIDIA/cuda-q-academic/blob/main/quantum-applications-to-finance/03_qchop.ipynb), demonstrate how quantum computing can be used outright to solve optimization problems. This notebook demonstrates a slightly different approach. Rather than considering QPUs as the tool to produce the final answer, it demonstrates how quantum can be used to enhance the effectiveness of leading classical methods.  

The benefits of such an approach were highlighted in [Scaling advantage with quantum-enhanced memetic tabu search for LABS](https://arxiv.org/html/2511.04553v1).  This notebook, co-created with the authors of the paper, will allow you to explore the findings of their research and write your own CUDA-Q code that builds a representative quantum-enhanced workflow for solving the LABS problem. Moreover, it will introduce advancements in counteradiabatic optimization techniques on which reduce the quantum resources required to run on a QPU.

**Prerequisites:** This lab assumes you have a basic knowledge of quantum computing, including operators, gates, etc.  For a refresher on some of these topics, explore the [Quick start to Quantum](https://github.com/NVIDIA/cuda-q-academic/tree/main/quick-start-to-quantum) series.

**In this lab you will:**
* 1. Understand the LABS problem and its relation ot radar and communication applications.
* 2. Solve LABS classically with memetic tabu search and learn about the limitations of such methods.
* 3. Code a couteradiabatic algorithm using CUDA-Q to produce approximate solutions to the LABS problem.
* 4. Use the CUDA-Q results to seed your tabu search and understand the potential benefits of this approach.


**Terminology you will use:**
* Low autocorrelation of binary sequences (LABS)
* counteradiabatic optimization
* memetic-tabu search

**CUDA-Q Syntax you will use:**
* cudaq.sample()
* @cudaq.kernel
* ry(), rx(), rz(), x(), h() 
* x.ctrl()

Run the code below to initialize the libraries you will need.

In [2]:
import cudaq
import numpy as np
from math import floor
import auxiliary_files.labs_utils as utils

## The LABS problem and applications

The **Low Autocorrelation Binary Sequences (LABS)** problem is fundamental to many applications, but originated with applications to radar. 

Consider a radar that monitors airport traffic.  The radar signal sent to detect incoming planes must have as much range as possible to ensure safe approaches are planned well in advance.  The range of a radar signal can be increased by sending a longer pulse.  However, in order to differentiate between multiple objects, pulses need to be short to provide high resolution. So, how do you handle situations where you need both?

One solution is a technique called pulse compression.  The idea is to send a long signal, but vary the phase at regular intervals such that the resolution is increased. Generally, the initial signal will encode a binary sequence of phase shifts, where each interval corresponds to a signal with a 0 or 180 degree phase shift. 

The tricky part is selecting an optimal encoding sequence.  When the signal returns, it is fed into a matched filter with the hope that a singular sharp peak will appear, indicating clear detection.  The autocorrelation of the original signal, or how similar the signal is to itself,  determines if a single peak or a messier signal with sidelobes will be detected. A signal should have high autocorrelation when overlayed on top of itself, but low autocorrelation when shifted with a lag. 

Consider the image below.  The signal on the left has a crisp single peak while the single on the right produces many undesirable sidelobes which can inhibit clear detection.  

<img src="images/quantum_enhanced_optimization_LABS/radar.png" width="800">


So, how do you select a good signal?   This is where LABS comes in, defining these questions as a binary optimization problem. Given a binary sequence of length $N$, $(s_1 \cdots s_N) \in {\pm 1}^N$, the goal is to minimize the following objective function.

$$ E(s) = \sum_{k=1}^{N-1} C_k^2 $$

Where $C_k$ is defined as. 

 $$C_k= \sum_{i=1}^{N-k} s_is_{i+k}$$


So, each $C_k$ computes how similar the original signal is to the shifted one for each offset value $k$.  To explore this more, try the interactive widget linked [here](https://nvidia.github.io/cuda-q-academic/interactive_widgets/labs_visualization.html).  See if you can select a very good and very poor sequence and see the difference for the case of $N=7$.

## Classical Solution of the LABS problem

The LABS problem is tricky to solve for a few reasons. First, the configuration space grows exponentially.  Second, underlying symmetries of the problem result in many degeneracies in the optimization landscape severely inhibiting local search methods. 

<div style="background-color: #f9fff0; border-left: 6px solid #76b900; padding: 15px; border-radius: 4px; box-shadow: 0px 2px 4px rgba(0,0,0,0.1);">
    <h3 style="color: #76b900; margin-top: 0; margin-bottom: 10px;">Exercise 1:</h3>
    <p style="font-size: 16px; color: #333;">
Using the widget above, try to find some of the symmetries for the LABS problem. That is, for a fixed bitstring length, can you find patterns to produce the same energy with different pulse patterns. 
</div>

The best known performance for a classical optimization technique is Memetic Tabu search (MTS) which exhibits a scaling of $O(1.34^N)$.  The MTS algorithm is depicted below.  It begins with a randomly selected population of bitstrings and finds the best solution from them.  Then, a child is selected by sampling directly from or combining multiple bitstrings from the population.  The child is mutated with probability $p_{mutate}$ and then input to a tabu search, which performs a modified greedy local search starting from the child bitstring.  If the result is better than the best in the population, it is updated as the new leader and randomly replaces a  bitstring in the population.


<img src="images/quantum_enhanced_optimization_LABS/mts_algorithm.png" width="500">

Such an approach is fast, parallelizable, and allows for exploration with improved searching of the solution landscape.  

<div style="background-color: #f9fff0; border-left: 6px solid #76b900; padding: 15px; border-radius: 4px; box-shadow: 0px 2px 4px rgba(0,0,0,0.1);">
    <h3 style="color: #76b900; margin-top: 0; margin-bottom: 10px;">Exercise 2:</h3>
    <p style="font-size: 16px; color: #333;">
Before exploring any quantum approach, get a sense for how MTS works by coding it yourself based generally on the figure above. Algorithms for the combine and mutate steps are provided below as used in the paper. You may need to research more specific details of the process, especially for how tabu search is performed. The MTS procedure should output and optimal bitstring and its energy.  Also, write a function to visualize the results including the energy distribution of the final population.
</div>



<img src="images/quantum_enhanced_optimization_LABS/combine_mutate.png" width="400">



In [106]:
def Combine(p1, p2):
    n = len(p1)
    k = np.random.randint(1, n)
    return np.concatenate([p1[:k], p2[k:]])

def Mutate(seq, p_mut):
    seq = seq.copy()
    for i in range(len(seq)):
        if (np.random.random() < p_mut):
            seq[i] *= -1
    return seq

def flip(seq, i):
    s_new = seq.copy()
    s_new[i] *= -1
    return s_new

def energy(seq):
    # Compute E(S) = Σ C_k²
    E = 0
    n = len(seq)
    for k in range(1, n):
        C_k = np.sum(seq[:n-k] * seq[k:])
        E += C_k * C_k
    return E

    
def TabuSearch(seq, max_iter, min_tabu, max_tabu):
    n = len(seq)
    tabu_list = np.zeros(n)
    
    best_seq_ts = seq.copy()
    best_energy_ts = energy(best_seq_ts)
    
    # pivot ← seq
    pivot = seq.copy()
    pivot_energy = energy(pivot)
    
    for t in range(1, max_iter + 1):
        best_i = -1
        best_flip_energy = float('inf')
        best_candidates = []
        
        for i in range(n):
            if tabu_list[i] < t:
                flipped = flip(pivot, i)
                flip_energy = energy(flipped)
                
                if flip_energy < best_flip_energy:
                    best_flip_energy = flip_energy
                    best_candidates = [i]
                elif flip_energy == best_flip_energy:
                    # Randomly choose one if there are multiple
                    best_candidates.append(i)
                    
        if best_candidates:
            best_i = np.random.choice(best_candidates)
        else:
            # If all moves are tabu, pick the least recently tabu'd
            best_i = np.argmin(tabu_list)
            best_flip_energy = energy(flip(pivot, best_i))
        
        # pivot ← flip(pivot, i)
        pivot = flip(pivot, best_i)
        pivot_energy = best_flip_energy
        
        tabu_list[best_i] = t + np.random.randint(min_tabu, max_tabu + 1)
        
        if pivot_energy < best_energy_ts:
            best_seq_ts = pivot.copy()
            best_energy_ts = pivot_energy
    
    return best_seq_ts

    

def MTS(population):

    # stop the while!
    iteration = 0
    max_iterations = 1000
    
    Kp = len(population)     
    Np = len(population[0])
    for i in range(Kp):
        population[i] = TabuSearch(population[i], int(max_iterations // K), 1, max(int(np.sqrt(Np)), 2))

    best_seq = population[0].copy()
    best_energy = energy(best_seq)
    for p in population:
        if energy(p) < best_energy:
            best_seq = p.copy()
            best_energy = energy(p)

    history = [best_energy]
    
    while (best_energy > target_e and iteration < max_iterations):
        iteration += 1
        if (np.random.rand() < p_comb):
            p1_i = np.random.randint(0, Kp)
            p2_i = np.random.randint(0, Kp)
            child = Combine(population[p1_i], population[p2_i])
        else:
            child_i = np.random.randint(0, Kp)
            child = population[child_i].copy()
            
        child = Mutate(child, p_mut)
        child = TabuSearch(child, int(max_iterations // K), 1, max(int(np.sqrt(Np)), 2))
        child_energy = energy(child)
        if(best_energy > child_energy):
            best_seq = child.copy()
            best_energy = child_energy

        history.append(best_energy)
            
        random_idx = np.random.randint(0, K)
        population[random_idx] = child
        
        # Progress
        if iteration % 20 == 0:
            print(f"Iteration {iteration}: Best Energy = {best_energy}")
    
    return best_seq, best_energy, history


In [107]:
N = 20
seq = np.random.choice([-1, 1], size = N)
    
print(f"Initial sequence energy: {energy(seq)}")


max_iter = 10 * N
min_tabu = 1
max_tabu = max(int(np.sqrt(N)), 2)

best_seq = TabuSearch(seq, max_iter, min_tabu, max_tabu)
    
print(f"After Tabu-Search energy: {energy(best_seq)}")
print(f"Best sequence: {best_seq.tolist()}")

Initial sequence energy: 90
After Tabu-Search energy: 34
Best sequence: [-1, 1, 1, -1, -1, 1, 1, -1, -1, -1, -1, 1, 1, 1, 1, 1, -1, 1, -1, 1]


In [97]:
K = 2
target_e = 25
p_comb = 0.7
p_mut = 1/N
N = 20
population = [np.random.choice([-1, 1], size = N) for _ in range(K)]
best_seq, best_energy, history = MTS(population)
print(best_seq, best_energy)

Iteration 20: Best Energy = 26
Iteration 40: Best Energy = 26
Iteration 60: Best Energy = 26
Iteration 80: Best Energy = 26
Iteration 100: Best Energy = 26
Iteration 120: Best Energy = 26
Iteration 140: Best Energy = 26
Iteration 160: Best Energy = 26
Iteration 180: Best Energy = 26
Iteration 200: Best Energy = 26
Iteration 220: Best Energy = 26
Iteration 240: Best Energy = 26
Iteration 260: Best Energy = 26
Iteration 280: Best Energy = 26
Iteration 300: Best Energy = 26
Iteration 320: Best Energy = 26
Iteration 340: Best Energy = 26
Iteration 360: Best Energy = 26
Iteration 380: Best Energy = 26
Iteration 400: Best Energy = 26
Iteration 420: Best Energy = 26
Iteration 440: Best Energy = 26
Iteration 460: Best Energy = 26
Iteration 480: Best Energy = 26
Iteration 500: Best Energy = 26
Iteration 520: Best Energy = 26
Iteration 540: Best Energy = 26
Iteration 560: Best Energy = 26
Iteration 580: Best Energy = 26
Iteration 600: Best Energy = 26
Iteration 620: Best Energy = 26
Iteration 64

## Building a Quantum Enhanced Workflow

Despite the effectiveness of MTS, it still exhibits exponential scaling  $O(1.34^N)$ behavior and becomes intractable for large $N$.  Quantum computing provides a potential alternative method for solving the LABS problem because the properties of entanglement, interference, and superpositon may allow for a better global search.  Recent demonstrations have even produced evidence that the quantum approximate optimization algorithm (QAOA) can be used to reduce the scaling of the LABS problem to $O(1.21^N)$ for $N$ between 28 and 40 with quantum minimum finding.

However, current quantum hardware limitations restrict solution to problems of greater than about $N=20$, meaning that it will be some time before quantum approaches can outperform the classical state of the art. It should also be noted that standard QAOA can struggle with LABS and require many layers to converge the parameters if other tricks are not employed.

The authors of [Scaling advantage with quantum-enhanced memetic tabu search for LABS](https://arxiv.org/html/2511.04553v1) cleverly explored an alternate path that combines quantum and classical approaches and might be able to provide a more near-term benefit.  Instead of expecting the quantum computer to solve the problem entirely, they asked how a quantum approach might enhance MTS.

The basic idea is that a quantum optimization routine could run first and the resulting state be sampled to produce a better population for MTS. Many such heuristics for defining the initial population are possible, but the rest of this notebook will explore their methodology, help you to build the workflow yourself, and allow you to analyze the benefits of their approach.

The first step of quantum enhanced MTS (QE-MTS) is to prepare a circuit with CUDA-Q that approximates the ground state of the Hamiltonian corresponding to the LABS problem. You could do this with any optimization algorithm such as QAOA or using an adiabatic approach.  (See the [Quantum Portfolio Optimization](https://github.com/NVIDIA/cuda-q-academic/blob/main/quantum-applications-to-finance/03_qchop.ipynb) CUDA-Q Academic lab for a detailed comparison of these two common approaches.)

The authors of this work opted for an adiabatic approach (More on why later). Recall that the goal of an adiabatic optimization is to begin with a Hamiltonian that has an easily prepared ground state ($H_i$). Then, the adiabatic Hamiltonian $H_{ad}$ can be constructed as $H_{ad}(\lambda) = (1-\lambda)H_i +\lambda H_f $, where $\lambda$ is a function of time and $H_f$ is the Hamiltonian representing a qubit encoding of the LABS problem. 

$$H_f = 2 \sum_{i=1}^{N-2} \sigma_i^z \sum_{k=1}^{\lfloor \frac{N-i}{2} \rfloor} \sigma_{i+k}^z 
+ 4 \sum_{i=1}^{N-3} \sigma_i^z \sum_{t=1}^{\lfloor \frac{N-i-1}{2} \rfloor} \sum_{k=t+1}^{N-i-t} \sigma_{i+t}^z \sigma_{i+k}^z \sigma_{i+k+t}^z$$

The authors also selected $H_i = \sum_i h^x_i \sigma^x_i $ which has an easily prepared ground state of $\ket{+}^{\otimes N}$.

The challenge for implementing the optimization procedure becomes selection of an operator that will quickly and accurately evolve to the ground state of $H_f$.  One approach is to use a so-called auxiliary countradiabatic (CD) term $H_{CD}$, which corrects diabatic transitions that jump out of the ground state during the evolution. The figure below demonstrates the benefit of using a CD correction.


<img src="images/quantum_enhanced_optimization_LABS/counteradiabatic.png" width="900">




An operator called the adiabatic gauge potential $A_{\lambda}$ is the ideal choice for the CD term as it suppresses all possible diabatic transitions, resulting in the following total system to evolve.

$$ H(\lambda) = H_{ad}(\lambda) + \lambda H_{CD} (\lambda) $$

$A(\lambda)$ is derrived from $H_{ad}(\lambda)$  (see paper for details) as it contains information about underlying physics of the problem. 

There is a problem though.  The $A(\lambda)$ term cannot be efficiently expressed exactly and needs to be approximated.  It also turns out that in the so called impulse regime, where the adiabatic evolution is very fast, $H_{cd} (\lambda)$ dominates $H_{ad}(\lambda)$, meaning that the final implementation corresponds to the operator $H(\lambda) = H^1_{cd}(\lambda)$ where  $H^1_{cd}(\lambda)$ is a first order approximation of $A(\lambda)$ (see equation 7 in the paper).

A final step is to use Trotterization to define the quantum circuit to apply $e^{-\theta (t) i H_{cd}}$. The details for this derivation are shown in the appendix of the paper. and result from equation B3 is shown below.  

$$
\begin{equation}
\begin{aligned}
U(0, T) = \prod_{n=1}^{n_{\text{trot}}} & \left[ \prod_{i=1}^{N-2} \prod_{k=1}^{\lfloor \frac{N-i}{2} \rfloor} R_{Y_i Z_{i+k}}\big(4\theta(n\Delta t)h_i^x\big) R_{Z_i Y_{i+k}}\big(4\theta(n\Delta t)h_{i+k}^x\big) \right] \\
& \times \prod_{i=1}^{N-3} \prod_{t=1}^{\lfloor \frac{N-i-1}{2} \rfloor} \prod_{k=t+1}^{N-i-t} \bigg( R_{Y_i Z_{i+t} Z_{i+k} Z_{i+k+t}}\big(8\theta(n\Delta t)h_i^x\big) \\
& \quad \times R_{Z_i Y_{i+t} Z_{i+k} Z_{i+k+t}}\big(8\theta(n\Delta t)h_{i+t}^x\big) \\
& \quad \times R_{Z_i Z_{i+t} Y_{i+k} Z_{i+k+t}}\big(8\theta(n\Delta t)h_{i+k}^x\big) \\
& \quad \times R_{Z_i Z_{i+t} Z_{i+k} Y_{i+k+t}}\big(8\theta(n\Delta t)h_{i+k+t}^x\big) \bigg)
\end{aligned}
\end{equation}
$$

It turns out that this implementation is more efficient than QAOA in terms of gate count. The authors calculated that for $N=67$, QAOA would require 1.4 million entangling gates while the CD approach derived here requires only 236 thousand entangling gates.


<div style="background-color: #f9fff0; border-left: 6px solid #76b900; padding: 15px; border-radius: 4px; box-shadow: 0px 2px 4px rgba(0,0,0,0.1);">
    <h3 style="color: #76b900; margin-top: 0; margin-bottom: 10px;">Exercise 3:</h3>
    <p style="font-size: 16px; color: #333;">
At first glance, this equation might looks quite complicated. However, observe the structure and note two "blocks" of terms.  Can you spot them?  

They are 2 qubit terms that look like $R_{YZ}(\theta)$ or $R_{ZY}(\theta)$.

As well as 4 qubit terms that look like $R_{YZZZ}(\theta)$, $R_{ZYZZ}(\theta)$, $R_{ZZYZ}(\theta)$, or $R_{ZZZY}(\theta)$.

Thankfully the authors derive a pair of circuit implementations for the two and four qubit terms respectively, shown in the figures below.

Using CUDA-Q, write a kernel for each which will be used later to construct the full implementation.

* Hint: Remember that the adjoint of a rotation gate is the same as rotating in the opposite direction. 

* Hint: You may also want to define a CUDA-Q kernel for an R$_{ZZ}$ gate.

* Hint: Implementing a circuit from a paper is a great place where AI can help accelerate your work.  If you have access to a coding assistant, feel free to use it here.
</div>

<img src="images/quantum_enhanced_optimization_LABS/kernels.png" width="1300">


In [11]:
@cudaq.kernel
def RZZ(theta:float , q0 : cudaq.qubit , q1 : cudaq.qubit):
    cx(q0,q1)
    rz(theta,q3)
    cx(q1,q0)

@cudaq.kernel
def RYZ(theta:float , q0 : cudaq.qubit , q1 : cudaq.qubit): 
    rx(math.pi/2 , q0)
    rz(theta , q0 , q1 )
    rx(-math.pi/2 , q0)

@cudaq.kernel
def RZY(theta:float , q0 : cudaq.qubit , q1 : cudaq.qubit):
    rx(math.pi/2 , q1)
    rz(theta , q0 , q1 )
    rx(-math.pi/2 , q1)

def RZZZZ(theta:float , q0 : cudaq.qubit , q1 : cudaq.qubit , q2 : cudaq.qubit ,q3 : cudaq.qubit ):
    cx(q0,q3)
    cx(q1,q3)
    cx(q2,q3)
    rz(theta,q3)
    cx(q2,q3)
    cx(q1,q3)
    cx(q0,q3)
    
    
@cudaq.kernel
def RYZZZ(theta:float , q0 : cudaq.qubit , q1 : cudaq.qubit , q2 : cudaq.qubit ,q3 : cudaq.qubit ):
    rx(math.pi/2 , q0)
    cx(q0,q3)
    cx(q1,q3)
    cx(q2,q3)
    rz(theta,q3)
    cx(q2,q3)
    cx(q1,q3)
    cx(q0,q3)
    rx(-math.pi/2 , q0)

@cudaq.kernel
def RZYZZ(theta:float , q0 : cudaq.qubit , q1 : cudaq.qubit , q2 : cudaq.qubit ,q3 : cudaq.qubit ):
    rx(math.pi/2 , q1)
    cx(q0,q3)
    cx(q1,q3)
    cx(q2,q3)
    rz(theta,q3)
    cx(q2,q3)
    cx(q1,q3)
    cx(q0,q3)
    rx(-math.pi/2 , q1)

@cudaq.kernel
def RZZYZ(theta:float , q0 : cudaq.qubit , q1 : cudaq.qubit , q2 : cudaq.qubit ,q3 : cudaq.qubit ):
    rx(math.pi/2 , q2)
    cx(q0,q3)
    cx(q1,q3)
    cx(q2,q3)
    rz(theta,q3)
    cx(q2,q3)
    cx(q1,q3)
    cx(q0,q3)
    rx(-math.pi/2 , q2)

@cudaq.kernel
def RZZZY(theta:float , q0 : cudaq.qubit , q1 : cudaq.qubit , q2 : cudaq.qubit ,q3 : cudaq.qubit ):
    rx(math.pi/2 , q3)
    cx(q0,q3)
    cx(q1,q3)
    cx(q2,q3)
    rz(theta,q3)
    cx(q2,q3)
    cx(q1,q3)
    cx(q0,q3)
    rx(-math.pi/2 , q3)

There are a few additional items we need to consider before completing the final implementation of the entire circuit.  One simplification we can make is that for our problem the $h_i^x$ terms are all 1 and any $h_b^x$ terms are 0, and are only there for generalizations of this model. 

The remaining challenge is derivation of the angles that are used to apply each of the circuits you defined above. These are obtained from two terms $\lambda(t)$ and $\alpha(t)$.  


The $\lambda(t)$ defines an annealing schedule and is generally a Sin function which slowly "turns on" the problem Hamiltonian.  For computing our angles, we need the derivative of $\lambda(t)$.

The $\alpha$ term is a bit trickier and is the solution to a set of differential equations which minimize the distance between $H^1_{CD}(\lambda)$ and $A(\lambda)$.  The result is 

$$\alpha(t) = \frac{-\Gamma_1(t)}{\Gamma_2(t)} $$

Where $\Gamma_1(t)$ and $\Gamma_2(t)$ are defined in equations 16 and 17 of the paper and essentially depend on the structure of the optimization problem.  Curious learners can look at the functions in `labs_utils.py`  to see how these are computed, based on the problem size and specific time step in the Trotter process. 


<div style="background-color: #f9fff0; border-left: 6px solid #76b900; padding: 15px; border-radius: 4px; box-shadow: 0px 2px 4px rgba(0,0,0,0.1);">
    <h3 style="color: #76b900; margin-top: 0; margin-bottom: 10px;">Exercise 4:</h3>
    <p style="font-size: 16px; color: #333;">
The `compute_theta` function, called in the following cells, requires all indices of the two and four body terms. These will be used again in our main kernel to apply the respective gates.  Use the products in the formula below to finish the function in the cell below.  Save them as `G2` and `G4` where each is a list of lists of indices defining the two and four term interactions. As you are translating an equation to a set of loops, this is a great opportunity to use an AI coding assistant.

$$
\begin{equation}
\begin{aligned}
U(0, T) = \prod_{n=1}^{n_{\text{trot}}} & \left[ \prod_{i=1}^{N-2} \prod_{k=1}^{\lfloor \frac{N-i}{2} \rfloor} R_{Y_i Z_{i+k}}\big(4\theta(n\Delta t)h_i^x\big) R_{Z_i Y_{i+k}}\big(4\theta(n\Delta t)h_{i+k}^x\big) \right] \\
& \times \prod_{i=1}^{N-3} \prod_{t=1}^{\lfloor \frac{N-i-1}{2} \rfloor} \prod_{k=t+1}^{N-i-t} \bigg( R_{Y_i Z_{i+t} Z_{i+k} Z_{i+k+t}}\big(8\theta(n\Delta t)h_i^x\big) \\
& \quad \times R_{Z_i Y_{i+t} Z_{i+k} Z_{i+k+t}}\big(8\theta(n\Delta t)h_{i+t}^x\big) \\
& \quad \times R_{Z_i Z_{i+t} Y_{i+k} Z_{i+k+t}}\big(8\theta(n\Delta t)h_{i+k}^x\big) \\
& \quad \times R_{Z_i Z_{i+t} Z_{i+k} Y_{i+k+t}}\big(8\theta(n\Delta t)h_{i+k+t}^x\big) \bigg)
\end{aligned}
\end{equation}
$$

</div>


In [12]:
def get_interactions(N):
    """
    Generates the interaction sets G2 and G4 based on the loop limits in Eq. 15.
    Returns standard 0-based indices as lists of lists of ints.
    
    Args:
        N (int): Sequence length.
        
    Returns:
        G2: List of lists containing two body term indices
        G4: List of lists containing four body term indices
    """
    
    G2 = [ ]
    G4 = [ ]

    for i in range(N-2):
        for k in range(1, (N-i) // 2):
            G2.append([i , i + k])

    for i in range(N-3):
        for t in range(1, (N-i-1) // 2):
            for k in range(t + 1 , N-i-t):
                G4.append([i , i + t , i + k , i + k + t])


    return G2, G4



<div style="background-color: #f9fff0; border-left: 6px solid #76b900; padding: 15px; border-radius: 4px; box-shadow: 0px 2px 4px rgba(0,0,0,0.1);">
    <h3 style="color: #76b900; margin-top: 0; margin-bottom: 10px;">Exercise 5:</h3>
    <p style="font-size: 16px; color: #333;">
You are now ready to construct the entire circuit and run the counteradiabatic optimization procedure. The final kernel needs to apply Equation 15 for a specified total evolution time $T$ and the `n_steps` number of Trotter steps.  It must also take as input, the indices for the two and four body terms and the thetas to be applied each step, as these cannot be computed within a CUDA-Q kernel.

The helper function `compute_theta` computes the theta parameters for you, using a few additional functions in accordance with the equations defined in the paper.
</div>


In [40]:
def split_G2(G2):
    Gi, Gj = [], []
    for pair in G2:
        i, j = pair[0], pair[1]
        if i != j:
            Gi.append(int(i))
            Gj.append(int(j))
    return Gi, Gj

def split_G4(G4):
    Ga, Gb, Gc, Gd = [], [], [], []
    for quad in G4:
        a,b,c,d = quad
        if len({a,b,c,d}) == 4:
            Ga.append(int(a)); Gb.append(int(b)); Gc.append(int(c)); Gd.append(int(d))
    return Ga, Gb, Gc, Gd


@cudaq.kernel
def trotterized_circuit(N: int,
                        G2_i: list[int],
                        G2_j: list[int],
                        G4_a: list[int],
                        G4_b: list[int],
                        G4_c: list[int],
                        G4_d: list[int],
                        steps: int,
                        thetas: list[float]):

    reg = cudaq.qvector(N)
    h(reg)

    for s in range(steps):
        th = thetas[s]

        for t in range(len(G2_i)):
            i = G2_i[t]
            j = G2_j[t]

            # RYZ(th): Y on i
            rx(math.pi/2 , reg[i])
            rz(th , reg[i] , reg[j] )
            rx(-math.pi/2 , reg[j])


            # RZY(th): Y on j
            rx(math.pi/2 , reg[j])
            rz(th , reg[i] , reg[j] )
            rx(-math.pi/2 , reg[j])

        for t in range(len(G4_a)):
            a = G4_a[t]; b = G4_b[t]; c = G4_c[t]; d = G4_d[t]

            # inline RZZZZ(th) parity ladder to d, with Y basis change on each qubit
            # Y on a
            rx(math.pi/2.0, reg[a])
            cx(reg[a], reg[d]); cx(reg[b], reg[d]); cx(reg[c], reg[d])
            rz(th, reg[d])
            cx(reg[c], reg[d]); cx(reg[b], reg[d]); cx(reg[a], reg[d])
            rx(-math.pi/2.0, reg[a])

            # Y on b
            rx(math.pi/2.0, reg[b])
            cx(reg[a], reg[d]); cx(reg[b], reg[d]); cx(reg[c], reg[d])
            rz(th, reg[d])
            cx(reg[c], reg[d]); cx(reg[b], reg[d]); cx(reg[a], reg[d])
            rx(-math.pi/2.0, reg[b])

            # Y on c
            rx(math.pi/2.0, reg[c])
            cx(reg[a], reg[d]); cx(reg[b], reg[d]); cx(reg[c], reg[d])
            rz(th, reg[d])
            cx(reg[c], reg[d]); cx(reg[b], reg[d]); cx(reg[a], reg[d])
            rx(-math.pi/2.0, reg[c])

            # Y on d
            rx(math.pi/2.0, reg[d])
            cx(reg[a], reg[d]); cx(reg[b], reg[d]); cx(reg[c], reg[d])
            rz(th, reg[d])
            cx(reg[c], reg[d]); cx(reg[b], reg[d]); cx(reg[a], reg[d])
            rx(-math.pi/2.0, reg[d])

T = 1.0
n_steps = 1
dt = T / n_steps
N = 8

G2, G4 = get_interactions(N)

G2_i, G2_j = split_G2(G2)
G4_a, G4_b, G4_c, G4_d = split_G4(G4)

thetas = []
for step in range(1, n_steps + 1):
    t = step * dt
    thetas.append(utils.compute_theta(t, dt, T, N, G2, G4))  # float
result = cudaq.sample(trotterized_circuit, N, G2_i, G2_j, G4_a, G4_b, G4_c, G4_d, n_steps, thetas, shots_count=100)
print(result)


{ 00000100:1 00000111:1 00001110:1 00001111:1 00010100:2 00010101:1 00010110:1 00011010:1 00011100:1 00100101:1 00100111:1 00101000:1 00101001:1 00101010:3 00101111:1 00110000:1 00110010:2 00110011:1 00110110:1 00110111:2 00111000:1 00111010:1 00111110:1 01000010:1 01000011:1 01000101:2 01001000:1 01001110:1 01010001:1 01010011:2 01010101:1 01010110:1 01011010:1 01100011:1 01100100:2 01100110:1 01100111:1 01101010:2 01101011:2 01101100:2 01110110:1 01111000:2 01111011:1 01111100:4 01111101:1 10001001:1 10001101:1 10010010:1 10010110:1 10010111:2 10100000:1 10100010:1 10100111:1 10101110:1 10101111:2 10110001:1 10110011:1 10111000:2 10111010:1 10111011:1 10111101:1 11000001:1 11000011:1 11000100:1 11000110:1 11010000:2 11010010:1 11010011:1 11011010:1 11100101:3 11101100:1 11110101:1 11110111:3 11111001:2 11111100:2 }



## Generating Quantum Enhanced Results

Recall that the point of this lab is to demonstrate the potential benefits of running a quantum subroutine as a preprocessing step for classical optimization of a challenging problem like LABS. you now have all of the tools you need to try this for yourself.

<div style="background-color: #f9fff0; border-left: 6px solid #76b900; padding: 15px; border-radius: 4px; box-shadow: 0px 2px 4px rgba(0,0,0,0.1);">
    <h3 style="color: #76b900; margin-top: 0; margin-bottom: 10px;">Exercise 6:</h3>
    <p style="font-size: 16px; color: #333;">
Use your CUDA-Q code to prepare an initial population for your memetic search algorithm and see if you can improve the results relative to a random initial population.  If you are running on a CPU, you will need to run smaller problem instances. The code below sets up the problem

</div>


In [119]:
def correlated_random_pop(N: int, pop_size: int, p_copy: float = 0.7):
    pop = []
    for _ in range(pop_size):
        s = np.random.choice([-1, 1], size=N).astype(np.int8)
        for i in range(N - 1):
            if np.random.rand() < p_copy:
                s[i + 1] = s[i]
        pop.append(s)
    return pop

N = 10
pop_size = 50

quantum_pop = counts_to_quantum_pop(result, N=N, pop_size=pop_size, take_last_bits=True)

random_pop = correlated_random_pop(N=N, pop_size=pop_size, p_copy=0.7)

quantum_pop = np.array(quantum_pop)  
random_pop  = np.array(random_pop)  

seq_q, e_q, hist_q = MTS(quantum_pop)
seq_r, e_r, hist_r= MTS(random_pop)

print("Random best sequence:", e_q,"Quantum-like energy:", E_q)
print("Random best energy:", e_r, "Random energy:", E_r)

Random best sequence: 13 Quantum-like energy: [ 1 -1  1  1 -1 -1]
Random best energy: 13 Random energy: [ 1  1  1  1 -1  1]


The results clearly show that a population sampled from CUDA-Q results in an improved distribution and a lower energy final result. This is exactly the goal of quantum enhanced optimization.  To not necessarily solve the problem, but improve the effectiveness of state-of-the-art classical approaches. 

A few major caveats need to be mentioned here. First, We are comparing a quantum generated population to a random sample.  It is quite likely that other classical or quantum heuristics could be used to produce an initial population that might even beat the counteradiabatic method you used, so we cannot make any claims that this is the best. 

Recall that the point of the counteradiabatic approach derived in the paper is that it is more efficient in terms of two-qubit gates relative to QAOA. The benefits of this regime would only truly come into play in a setting (e.g. larger problem instance) where it is too difficult to produce a good initial population with any know classical heuristic, and the counteradiabatic approach is more efficiently run on a QPU compared to alternatives.

We should also note that we are comparing a single sample of each approach.  Maybe the quantum sample got lucky or the randomly generated population was unlucky and a more rigorous comparison would need to repeat the analysis many times to draw any confidently conclusions.  

The authors of the paper discuss all of these considerations, but propose an analysis that is quite interesting related to the scaling of the technique. Rather than run large simulations ourselves, examine their results below. 


<img src="images/quantum_enhanced_optimization_LABS/tabu_search_results.png" width="900">

The authors computed replicate median (median of solving the problem repeated with same setup) time to solutions (excluding time to sample from QPU) for problem sizes $N=27$ to $N=37$. Two interesting conclusions can be drawn from this. First, standard memetic tabu search (MTS) is generally faster than quantum enhanced (QE) MTS.  But there are two promising trends. For larger problems, the QE-MTS experiments occasionally have excellent performance with times to solution much smaller than all of the MTS data points.  These outliers indicate there are certain instances where QE-MTS could provide much faster time-to-solution. 

More importantly, if a line of best fit is calculated using the median of each set of medians, the slope of the QE-MTS line is smaller than the MTS!  This seems to indicate that QE solution of this problem scales $O(1.24^N)$ which is better than the best known classical heuristic ($O(1.34^N)$) and the best known quantum approach (QAOA - $O(1.46^N)$).

For problems of size of $N=47$ or greater, the authors anticipate that QE-MTS could be a promising technique and produce good initial populations that are difficult to obtain classically. 

The study reinforces the potential of hybrid workflows enhanced by quantum data such that a classical routine is still the primary solver, but quantum computers make it much more effective.  Future work can explore improvements to both the quantum and classical sides, such as including GPU accelerated memetic search on the classical side.

## Self-validation To Be Completed for Phase 1

In this section, explain how you verified your results. Did you calculate solutions by hand for small N? Did you create unit tests? Did you cross-reference your Quantum energy values against your Classical MTS results? Did you check known symmetries?


I added a normal energy calculation to make sure I am optimizing the energy. 

$$ E(s) = \sum_{k=1}^{N-1} C_k^2 $$

Also checked if the energies are corrrect:

In [122]:
S = seq_q  

print(f"Check 1: E(S)={energy(S)}, E(-S)={energy(-S)} → {'✓' if energy(S)==energy(-S) else '✗'}")

print(f"Check 2: E(S)={energy(S)}, E(S[::-1])={energy(S[::-1])} → {'✓' if energy(S)==energy(S[::-1]) else '✗'}")

S = seq_r

print(f"Check 1: E(S)={energy(S)}, E(-S)={energy(-S)} → {'✓' if energy(S)==energy(-S) else '✗'}")

print(f"Check 2: E(S)={energy(S)}, E(S[::-1])={energy(S[::-1])} → {'✓' if energy(S)==energy(S[::-1]) else '✗'}")

Check 1: E(S)=13, E(-S)=13 → ✓
Check 2: E(S)=13, E(S[::-1])=13 → ✓
Check 1: E(S)=13, E(-S)=13 → ✓
Check 2: E(S)=13, E(S[::-1])=13 → ✓
